In [98]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [99]:
df_num=pd.read_csv('numerical.csv')

df_cat=pd.read_csv('categorical.csv')
targets=pd.read_csv('target.csv')

In [100]:
df_cat.shape

(95412, 22)

In [101]:
df_cat

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,other,27,H,M,3,L,G,C,C,2,...,0,2,96,2,96,2,96,2,96,2
95408,TX,24,H,M,3,L,F,A,C,1,...,50,1,96,3,96,3,96,3,96,3
95409,MI,30,H,M,3,L,E,B,C,3,...,38,1,96,3,95,1,96,10,94,10
95410,CA,24,H,F,2,L,F,A,C,1,...,40,5,90,11,96,8,97,1,86,12


In [102]:
X=pd.concat([df_num, df_cat], axis=1)

In [103]:
targets

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
95407,0,0.0
95408,0,0.0
95409,0,0.0
95410,1,18.0


In [104]:
y=targets['TARGET_B']

In [105]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [106]:
train_num=X_train.select_dtypes(include=np.number)
train_cat=X_train.select_dtypes(include=np.object)

test_num=X_test.select_dtypes(include=np.number)
test_cat=X_test.select_dtypes(include=np.object)



In [107]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(train_num)
train_num_scaled = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns, index=train_num.index)
test_num_scaled = pd.DataFrame(transformer.transform(test_num), columns=test_num.columns, index=test_num.index)

In [108]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_cat)
column_name = encoder.get_feature_names_out(train_cat.columns)

train_encoded = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=column_name, index=train_cat.index)
test_encoded = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=column_name, index=test_cat.index)

In [109]:
train_all=pd.concat([train_encoded, train_num_scaled, y_train], axis=1)
X_test=pd.concat([test_encoded, test_num_scaled], axis=1)

In [110]:
no = train_all[train_all['TARGET_B']==0]
yes = train_all[train_all['TARGET_B']==1]

In [111]:
from sklearn.utils import resample

yes_oversampled = resample(yes, replace=True, n_samples = len(no),)

In [112]:
train_oversampled = pd.concat([no, yes_oversampled], axis=0)

In [113]:
train_oversampled.TARGET_B.value_counts()

0    72486
1    72486
Name: TARGET_B, dtype: int64

In [114]:
X_train_over = train_oversampled.drop('TARGET_B', axis = 1)
y_train_over = train_oversampled['TARGET_B']

In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=6)

In [120]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import precision_score

def model_evaluation(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    predictions=model.predict(X_test)
    cm=confusion_matrix(y_test, predictions)
    print( str(model.base_estimator_)[:-2] + 'score: ', model.score(X_test, y_test))
    print( str(model.base_estimator_)[:-2] + 'precision: ', precision_score(y_test, predictions))
    print( str(model.base_estimator_)[:-2] + 'recall: ', recall_score(y_test, predictions))
    print( str(model.base_estimator_)[:-2] + 'f1: ', f1_score(y_test, predictions))
    print('\n')
    print(cm)
   
    return model

In [119]:
model_evaluation(clf, X_train_over, y_train_over, X_test, y_test)

DecisionTreeClassifierscore:  0.6251113556568674
DecisionTreeClassifierprecision:  0.07334997226844149
DecisionTreeClassifierrecall:  0.529
DecisionTreeClassifierf1:  0.1288358499756454


[[11400  6683]
 [  471   529]]
0.38


RandomForestClassifier(max_depth=6)

## Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

kbest = SelectKBest(chi2, k=30).fit_transform(X, y)

In [ ]:
selected = pd.DataFrame(kbest)
selected.head()

In [ ]:
model = SelectKBest(chi2, k=30).fit(X, y)
df = pd.DataFrame(data = model.scores_, columns = ['score'])
df['Column'] = X.columns
print(df.sort_values(by = ['score'], ascending = False).head(30))

In [ ]:
cols = df.sort_values(by = ['score'], ascending = False).head(30)['Column']
cols

In [ ]:
selected.columns = cols
selected.head()

In [ ]:
cols

In [ ]:
y_train.value_counts()

In [ ]:
X_train=X_train[cols]
X_test=X_test[cols]

In [ ]:
clf = RandomForestClassifier(max_depth=6,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

> No much improvement

## RFE

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
#model = LinearRegression()

#rfe = RFE(model, n_features_to_select=30)

#rfe.fit(X, y)

In [ ]:
df2 = pd.DataFrame(data = rfe.ranking_, columns=['Rank'])
df2['Column_name'] = pd.DataFrame(X).columns
df2=df2[df2['Rank']<21].head(50)

In [ ]:
df2.Column_name.unique()

In [ ]:
cols2=df2['POP901', 'POP903', 'POP90C1', 'POP90C2', 'POP90C3', 'ETH1',
       'ETH10', 'AGE905', 'AGE906', 'HHAGE3', 'DW1', 'DW3', 'DW4', 'DW5',
       'HU3', 'HU4', 'HHD1', 'HHD4', 'HHD8', 'HHD9', 'ETHC1', 'ETHC2',
       'RHP1', 'RHP2', 'RHP3', 'RHP4', 'HUPA1', 'HUPA2', 'HUPA3', 'IC11',
       'IC13', 'IC20', 'IC22', 'MC1', 'MC2', 'TPE3', 'TPE4', 'TPE5',
       'TPE6', 'RAMNTALL', 'NGIFTALL', 'CARDGIFT', 'MINRAMNT', 'MAXRAMNT',
       'LASTGIFT', 'TIMELAG', 'ODATEW_MM', 'MINRDATE_YR', 'FIRSTDATE_YR']

In [ ]:
X_rfe = X.loc[:, cols2]

In [ ]:
cols3 = X_rfe.columns.tolist()

In [ ]:
y = df['TARGET_B']
X=X_rfe

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
trainset = pd.concat([X_train, y_train], axis=1)

category_0 = trainset[trainset['TARGET_B']==0].sample(len(trainset[trainset['TARGET_B']==1]))
print(category_0.shape)

category_1 = trainset[trainset['TARGET_B']== 1 ]
trainset_new = pd.concat([category_0, category_1], axis = 0)
trainset_new = trainset_new.sample(frac =1) 
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']

print(X_train.shape)

In [ ]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)


In [ ]:
clf = RandomForestClassifier(max_depth=6,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

> Same results as others feature selection.

## Variance Threshold

In [72]:
from sklearn.feature_selection import VarianceThreshold

In [73]:
var_threshold = 0.04
sel = VarianceThreshold(threshold=(var_threshold))

In [74]:
sel = sel.fit(train_num_scaled)

In [75]:
sel.get_support()
var_list=list(sel.get_support())
droplist_var=[col[0] for col in zip(train_num_scaled.columns, var_list) if col[1] == False]
print(droplist_var)
len(droplist_var)

['TCODE', 'AGE', 'HIT', 'MALEMILI', 'MALEVET', 'VIETVETS', 'WWIIVETS', 'LOCALGOV', 'STATEGOV', 'FEDGOV', 'POP901', 'POP902', 'POP903', 'POP90C4', 'POP90C5', 'ETH2', 'ETH3', 'ETH4', 'ETH5', 'ETH6', 'ETH7', 'ETH8', 'ETH9', 'ETH10', 'ETH11', 'ETH12', 'ETH13', 'ETH14', 'ETH15', 'ETH16', 'AGE901', 'AGE902', 'AGE903', 'AGE904', 'AGE905', 'AGE906', 'AGE907', 'CHIL1', 'CHIL2', 'CHIL3', 'AGEC1', 'AGEC2', 'AGEC3', 'AGEC4', 'AGEC5', 'AGEC6', 'AGEC7', 'CHILC1', 'CHILC2', 'CHILC3', 'CHILC4', 'CHILC5', 'HHAGE1', 'HHAGE2', 'HHAGE3', 'HHN1', 'HHN2', 'HHN3', 'HHN4', 'HHN5', 'HHN6', 'MARR1', 'MARR2', 'MARR3', 'MARR4', 'HHP1', 'HHP2', 'DW3', 'DW7', 'DW8', 'DW9', 'HV1', 'HV2', 'HV3', 'HV4', 'HU3', 'HU4', 'HHD1', 'HHD2', 'HHD3', 'HHD4', 'HHD5', 'HHD6', 'HHD7', 'HHD8', 'HHD9', 'HHD10', 'HHD11', 'HHD12', 'ETHC1', 'ETHC2', 'ETHC3', 'ETHC4', 'ETHC5', 'ETHC6', 'HVP6', 'HUR1', 'RHP1', 'RHP2', 'RHP3', 'RHP4', 'HUPA1', 'HUPA2', 'HUPA3', 'HUPA4', 'HUPA5', 'HUPA6', 'HUPA7', 'DMA', 'IC1', 'IC2', 'IC3', 'IC4', 'IC5', 

275

In [76]:
model_evaluation(clf, X_train_over.drop(droplist_var, axis=1), y_train_over, X_test.drop(droplist_var, axis=1), y_test)

ValueError: could not convert string to float: 'other'

### Prediction of whole dataframe

In [38]:
numerical = X.select_dtypes(include=np.number)
categorical = X.select_dtypes(include=np.object)

In [39]:
num_scaled = pd.DataFrame(transformer.transform(numerical), columns=train_num.columns, index=numerical.index)


In [40]:
encoded = pd.DataFrame(encoder.transform(categorical).toarray(), columns=column_name, index=categorical.index)

In [41]:
X_all=pd.concat([encoded, num_scaled], axis=1)

In [43]:
X_all_selected=X_all.drop(droplist_var, axis=1)

In [44]:
X['predicted donation']=clf.predict(X_all_selected)

In [46]:
X_predict_D_amount=X[X['predicted donation'] == True]

In [47]:
X_predict_D_amount

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,predicted donation
0,0,60.000000,5,9,0,0,39,34,18,10,...,12,92,8,94,2,95,12,89,11,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,2,91,11,92,7,95,12,90,1,1
3,0,70.000000,1,4,2,0,23,14,31,3,...,1,87,11,94,11,95,12,87,2,1
7,0,61.611649,2,9,1,0,34,36,12,3,...,2,95,7,95,8,95,8,94,1,1
12,0,72.000000,4,2,16,2,30,31,33,9,...,3,95,7,91,2,96,1,89,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95402,0,61.611649,5,9,0,0,20,21,63,7,...,2,95,1,95,12,95,12,95,1,1
95404,2,53.000000,1,9,0,0,24,31,28,9,...,6,94,1,95,8,95,10,93,10,1
95406,0,50.000000,6,6,10,2,44,26,46,25,...,3,96,2,91,3,96,2,90,3,1
95409,1,60.000000,5,9,0,0,18,46,20,7,...,1,96,3,95,1,96,10,94,10,1


## Start of Final Lab

In [77]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [121]:
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')
target = pd.read_csv('target.csv')

data = pd.concat([categorical,numerical],axis=1)

In [122]:
droplist_var

['TCODE',
 'AGE',
 'HIT',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'POP901',
 'POP902',
 'POP903',
 'POP90C4',
 'POP90C5',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW3',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU3',
 'HU4',
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC2',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HVP6',
 'HUR1',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUP

In [123]:
data2=data[droplist_var]

In [124]:
data=pd.concat([data2, target], axis=1)

In [125]:
data.shape

(95412, 277)

In [126]:
target_d = data[data['TARGET_B']==1]

In [127]:
target_d.shape

(4843, 277)

> Doing all the proccess of split, scalling and encoding again

In [128]:
X = target_d.drop(['TARGET_B','TARGET_D'],axis=1)
y = target_d['TARGET_D']

In [129]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [130]:

X_train_num = X_train.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(object)

X_test_num = X_test.select_dtypes(include = np.number)
X_test_cat = X_test.select_dtypes(object)


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_train_num)
X_scaled_train = pd.DataFrame(transformer.transform(X_train_num),columns=X_train_num.columns)

X_scaled_test = pd.DataFrame(transformer.transform(X_test_num),columns=X_test_num.columns)
X_scaled_test

,TCODE,AGE,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,ODATEW_MM,MINRDATE_YR,MAXRDATE_YR,FIRSTDATE_YR
0,0.000000,0.614458,0.000000,0.010417,0.291139,0.363636,0.333333,0.041667,0.18,0.083333,...,0.034483,0.044444,0.015075,0.044444,0.204545,0.039619,0.0,0.909091,0.909091,1.000000
1,0.000000,0.561586,0.000000,0.000000,0.569620,0.262626,0.525253,0.229167,0.08,0.033333,...,0.379310,0.006667,0.010050,0.033333,0.113636,0.010920,0.0,0.090909,0.818182,0.444444
2,0.000026,0.421687,0.025000,0.000000,0.708861,0.292929,0.535354,0.395833,0.04,0.000000,...,0.000000,0.011111,0.015075,0.044444,0.272727,0.022869,0.0,0.727273,0.818182,0.888889
3,0.000718,0.819277,0.054167,0.000000,0.759494,0.070707,0.757576,0.208333,0.06,0.050000,...,0.448276,0.004444,0.001005,0.011111,0.590909,0.004877,0.0,0.181818,0.545455,0.444444
4,0.025691,0.493976,0.000000,0.000000,0.392405,0.282828,0.323232,0.083333,0.08,0.016667,...,0.137931,0.006667,0.001005,0.008889,0.113636,0.005001,0.0,0.818182,0.818182,0.944444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,0.000000,0.638554,0.000000,0.000000,0.417722,0.252525,0.363636,0.187500,0.16,0.016667,...,0.068966,0.011111,0.010050,0.033333,0.500000,0.021007,0.0,0.636364,0.727273,0.833333
965,0.000000,0.561586,0.000000,0.000000,0.405063,0.434343,0.232323,0.145833,0.08,0.033333,...,0.241379,0.011111,0.010050,0.022222,0.068182,0.021938,0.0,0.545455,0.818182,0.777778
966,0.025691,0.313253,0.004167,0.000000,0.506329,0.151515,0.626263,0.020833,0.08,0.033333,...,0.137931,0.011111,0.020101,0.055556,0.250000,0.030686,0.0,0.545455,0.909091,0.777778
967,0.000026,0.710843,0.054167,0.010417,0.417722,0.333333,0.232323,0.145833,0.10,0.033333,...,0.103448,0.011111,0.010050,0.028889,0.250000,0.017843,0.0,0.363636,0.818182,0.666667


In [131]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first').fit(X_train_cat)

encoded_train = encoder.transform(X_train_cat).toarray()
encoded_train = pd.DataFrame(encoded_train)

encoded_test = encoder.transform(X_test_cat).toarray()
encoded_test = pd.DataFrame(encoded_test)

In [132]:
train_scaled = pd.concat([encoded_train,X_scaled_train],axis=1)
test_scaled = pd.concat([encoded_test,X_scaled_test],axis=1)

# Regressor and prediction of average donation given

In [133]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [136]:
DT = DecisionTreeRegressor()
LR = LinearRegression()
RF = RandomForestRegressor()

model_pipeline = [DT, LR, RF]
model_names = ['Regression Tree', 'Linear Regression', 'Random Forest']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, train_scaled, y_train, cv=10))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': -0.17952309951015627, 'Linear Regression': 0.35948593475465074, 'Random Forest': 0.5090618275095516}


In [137]:
val_scores = {}
i=0
for model in model_pipeline:
    model.fit(train_scaled, y_train)
    val_scores[model_names[i]] = model.score(test_scaled,y_test)
    i = i+1
print(val_scores)

{'Regression Tree': -0.049746965322257086, 'Linear Regression': 0.22424080377678424, 'Random Forest': 0.38938028176393014}


In [138]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt

LR =RandomForestRegressor().fit(train_scaled, y_train)
pred = LR.predict(test_scaled)

In [139]:
r2 = r2_score(y_test,pred)
mse = mean_squared_error(y_test,pred)
mae = mean_absolute_error(y_test,pred)
rmse = sqrt(mse)

print('R2 Score:', r2)
print('Mean Squared Error:', mse)
print('Mean aboslute Error:', mae)
print('Root Mean Squared Error:',rmse)

R2 Score: 0.4057636020944442
Mean Squared Error: 113.24046521562434
Mean aboslute Error: 4.862240247678018
Root Mean Squared Error: 10.64145033421781


In [143]:

print('Average donation given is:', round(np.mean(pred),2))

Average donation given is: 16.43
